In [ ]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import json 
from pandas.io.json import json_normalize
import numpy as np

In [ ]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

url_content = requests.get(wiki_url).text
soup = BeautifulSoup(url_content, 'lxml')
bl = soup.find_all('table', {'class':'wikitable sortable'})


Borough = []
Neighbourhood = []
PostalCode = []


tab = bl[0].find_all('tr')    
for r in tab:
    cols = r.find_all('td')
    if len(cols) > 0:
        p = cols[0].text
        b = cols[1].text
        n = cols[2].text.rstrip()
        PostalCode.append(p)
        Borough.append(b)
        Neighbourhood.append(n)
            

df = pd.DataFrame()
nlist = {'PostalCode':PostalCode, 'Boroughs':Borough, 'Neighbours':Neighbourhood}
df = pd.DataFrame(nlist)
# Get rid with rows, where Boroughts are not assigned
df.drop(df[df.Boroughs == 'Not assigned'].index, inplace=True)

In [ ]:
cond=df['Neighbours']=='Not assigned'
df['Neighbours']=df['Boroughs'].where(cond,df['Neighbours'])

In [ ]:
df = df.sort_values(by='PostalCode', ascending=True)
df = df.groupby(['PostalCode','Boroughs'])['Neighbours'].apply(','.join).reset_index()

In [ ]:
df.head()

In [ ]:
df.shape